In [1]:
#Import all needed files
import csv
import sys, os, re
import time
import json

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.signal import butter, lfilter

import warnings

In [ ]:
#Import trial data
file_path = r"C:\Users\jgx7497\Documents\BME 390-3\Jay_Quarter2_Trial(in).csv"
trial_df = pd.read_csv(file_path)
